# 27-1. 프로젝트: KoChatGPT 업그레이드 하기


## Base model and Dataset for RLHF

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import pandas as pd
import numpy

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
input_txt = "바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."

In [4]:
tokens = tokenizer(input_txt).tokens()
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].numpy()

In [5]:
pd.options.display.max_columns = 40
pd.options.display.max_rows = 60
df = pd.DataFrame([tokens, input_ids[0]], index=["kogpt-2_tokens", "Input_IDs"])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
kogpt-2_tokens,▁바람,도,▁없는,▁공중에,▁수직,의,▁파,문을,▁내,이며,▁고,요,히,▁떨어지는,▁오동,잎은,▁누,구의,▁발자,취,▁입,니까,.
Input_IDs,10891,7235,9712,49207,14438,8143,9203,9941,9094,9639,9065,8084,8811,21215,34769,19985,9669,10139,21626,8408,9241,23775,389


In [6]:
#그리디 서치 디코딩
max_length=128
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_greedy = model.generate(input_ids, max_length=max_length, do_sample=False)
print(tokenizer.decode(output_greedy[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇다면 그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리요?"
"그건 무슨 소리


In [7]:
#이번엔 빔 서치 디코딩을 사용하고 n-gram 패널티까지 부과해보겠습니다.
input_ids = tokenizer(input_txt, return_tensors="pt")["input_ids"].to(device)
output_beam = model.generate(input_ids, max_length=max_length, num_beams=10, no_repeat_ngram_size=2,
                             do_sample=False)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까.'
"그렇지 않습니다."
"어떻게 된 일입니까?"
그녀는 고개를 갸웃거렸다.
"아니, 그게 무슨 말씀이신지 모르겠습니다만."
"무슨 말씀인지 알 수가 없군요."
아무런 대답도 하지 않은 채 그녀는 고개를 끄덕였다.
"그래, 알았어."
그녀의 눈에서 눈물이 주르륵 흘러내렸다.
그녀가 다시 입을 열었다.
"정말 죄송합니다, 고마워요, 고맙습니다"
"


입력 시퀀스와 별 상관 없어 보이는 긴 문단이 생성됩니다.
그럼에도 생성된 문단은 제법 맥락을 갖춘 듯 보입니다.
하지만 문장 간의 정합성이나 일관성은 다소 떨어지는 부분도 관찰됩니다.
이번엔 샘플링 기법까지 추가해 보겠습니다.

In [8]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, temperature=2.0, top_k=50)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까. 그거야 뭐가 <unk>니까?
그런데 그게 뭔지 모르겠어.
나한테도 알려주려나?
그럴 수도 있을 텐데.
난 그걸 모르고 있었어.
어떻게 얘기하지?
무슨 소리를 들었는지 기억 안 나?
아니야! 내가 기억해 줘. 고맙다는 인사나 하고.
그래도 괜찮아. 하고 싶지 않을 때면 아빠가 전화를 주셔.
이런 말 안 해? 내가 지금 뭘 하고


top_p 샘플링 기법도 사용해보겠습니다.

In [9]:
output_beam = model.generate(input_ids, max_length=max_length, num_beams=7, no_repeat_ngram_size=2,
                             do_sample=True, top_p=0.90)
print(tokenizer.decode(output_beam[0]))

바람도 없는 공중에 수직의 파문을 내이며 고요히 떨어지는 오동잎은 누구의 발자취 입니까."
"그렇지 않습니다. 저는 한 번도 저를 본 적이 없습니다."
그녀는 고개를 끄덕였다.
"무슨 말씀을 하시는 겁니까?"
"고맙습니다, 선생님."
"선생님, 저도 선생님을 뵙고 싶습니다. 선생님은 저와 함께 할 수 있는 유일한 분이십니다."
선생님은 고맙다는 인사말을 남기고 자리에서 일어섰다.
"저도 함께할 수 있기를 바랍니다!"
순간 고마운 생각이 머리를 스치고 지나갔다.



RLHF를 kogpt-2에 적용한다면
더 좋은 문장을 생성해내는 방법을 모델이 스스로 학습해낼 수 있으리라 기대해 볼 수 있습니다.
고도의 prompting은 어렵겠지만
현재 상태에서 특정 task에 fine-tuning했을 때보다는 성능이 한층 더 개선될 수 있지 않을까요?  
먼저 SFT를 시도할 initial 모델에 쓸 데이터셋을 살펴보겠습니다.

### 데이터셋 확인

In [10]:
import json 
data_path_1_SFT = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl' 
with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '불고기용 고기 한우에요?',
  'completion': "'저는 인공지능 챗봇이며, 직접적으로 식품에 관한 정보를 가지고 있지 않습니다. 하지만 일반적으로 불고기용 고기는 한우, 쇠고기, 돼지고기 등 다양한 종류의 고기를 사용합니다. 하지만 한우는 대표적인 고급 육류로 알려져 있기 때문에, 한우를 사용하는 경우도 많습니다. 알러지나 개별 건강 상태에 따라 다를 수 있으니 충분한 정보 수집 후에 선택해 주시기 바랍니다.",
  'tokens': 193},
 {'prompt': '쓰던 앱이 유료로 전환됐어',
  'completion': "'어떤 앱인지 모르기 때문에 정확한 답변을 드리기 어렵습니다. 하지만, 일반적으로 유료 전환된 앱은 기존 무료 앱에서 추가적인 기능이나 서비스를 제공하는 경우가 많습니다. 이 경우, 유료 전환 이전 가격이 매우 저렴하거나 무료인 경우가 많으므로, 앱을 계속 사용하려면 추가적인 비용을 지불해야 할 수 있습니다. 하지만, 유료 전환 이후에는 기존 사용자 또는 새로운 사용자 모두가 추가 기능과 높은 품질의 서비스를 이용할 수 있게 됩니다. 따라서, 앱 개발자는 유료 모델로 전환함으로써 앱의 수익을 증가시키고 더 나은 서비스를 제공할 수 있습니다.",
  'tokens': 288},
 {'prompt': '여친이랑 다툼',
  'completion': "'저는 인공지능이기 때문에 여친과의 다툼에 대해서는 조언을 드릴 수는 없습니다. 하지만 다투게 되면 상대방의 말을 경청하는 태도와 서로의 감정을 존중하는 것이 중요합니다. 논리적이고 차분한 대화를 통해 문제를 해결해보세요. 그리고 서로를 이해하고 서로의 의견을 수용하는 것이 중요합니다.",
  'tokens': 153}]

### Data Augmentation- SFT, RM, PPO 각 데이터셋 증강

In [11]:
# nltk에서 wordnet을 사용하기 위해 필요한 데이터 다운로드
import random
import nltk
from nltk.corpus import wordnet

In [12]:
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /aiffel/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /aiffel/nltk_data...


True

In [13]:
# Synonym Replacement: 문장 내의 단어를 그 단어의 동의어로 교체
def synonym_replacement(words, n=5):
    new_words = words.copy()
    random_word_list = list(set([word for word in words if word.isalnum()]))
    random.shuffle(random_word_list)
    num_replaced = 0
    for random_word in random_word_list:
        synonyms = get_synonyms(random_word)
        if len(synonyms) >= 1:
            synonym = random.choice(list(synonyms))
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n: 
            break
    return new_words

In [14]:
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
            synonym = "".join([char for char in synonym if char in ' qwertyuiopasdfghjklzxcvbnm'])
            synonyms.add(synonym)
    if word in synonyms:
        synonyms.remove(word)
    return list(synonyms)

In [15]:
# Random Deletion: 문장 내의 단어를 무작위로 삭제
def random_deletion(words, p=0.5): 
    if len(words) == 1: 
        return words
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: 
        return [random.choice(words)]
    else:
        return remaining


In [16]:
# Random Swap: 문장 내의 두 단어의 위치를 무작위로 바꿈.
def random_swap(sentence, n=5): 
    length = range(len(sentence))
    if len(sentence) < 2:  # Check if the sentence has less than 2 words
        return sentence
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1]
    return sentence

In [17]:
def augment_sentence(sentence, num_new_sentences=5):
    augmented_sentences = []
    words = sentence.split(' ')
    for _ in range(num_new_sentences):
        if random.uniform(0, 1) < 0.5:
            new_sentence_words = synonym_replacement(words)
        else:
            new_sentence_words = random_swap(words)
        augmented_sentences.append(' '.join(new_sentence_words))
    return augmented_sentences

In [21]:
augmented_data = []
for item in list_data_dict:
    augmented_data.append(item)
    prompt = item['prompt']
    completion = item['completion']
    for new_prompt in augment_sentence(prompt):
        new_data = item.copy()
        new_data['prompt'] = new_prompt
        augmented_data.append(new_data)
    for new_completion in augment_sentence(completion):
        new_data = item.copy()
        new_data['completion'] = new_completion
        augmented_data.append(new_data)
    if len(augmented_data) >= 15000:
        break

In [23]:
print(f"Filtered data size: {len(augmented_data)}")

Filtered data size: 15004


In [36]:
# 지정된 경로 및 파일명
file_path = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT_augmented.jsonl'

# 파일 쓰기 모드로 파일 열기
with open(file_path, 'w') as f:
    for item in augmented_data:
        f.write(json.dumps(item) + '\n')

다음으로 RM에 사용할 데이터셋을 살펴보겠습니다.

In [25]:
data_path_2_RM = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl'
with open(data_path_2_RM, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

10220


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?',
  'completion_0': 'Allow me to answer your question. I know that you are curious about me.',
  'completion_1': '번디는 다양한 인터뷰자들과 뉴스홍보 담당자들과의 면담 때 밝혔다.',
  'completion_2': '라이언에게 말했다.',
  'ranking': [2, 1, 0]},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?',
  'completion_0': '개포주공아파트는 다섯 단지로 이루어져 있습니다.',
  'completion_1': '이날 목송에서 구글상위노',
  'completion_2': '개포주공아파트는 총 27개 단지로 이루어져 있습니다.',
  'ranking': [2, 0, 1]},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?',
  'completion_0': 'The diameter of the Metallic domain is bigger than the Hyperonic domain.',
  'completion_1': '이 질문은 조금 불분명합니다. 김영삼 대통령이 후보 시절에 어떤 발언을 했고, 누가 그 발언을 문제삼았는지에 따라 답이 다를 수 있습니다.\\n\\n만약 김영삼 대통령이 후보 시절에 지역표심을 겨냥한 발언을 했다는 가정하에, 그 발언을 문제삼은 후보가 누구였는지를 대답하자면, 그 답은 이화선 당시 민주당 대통령 후보가 될 것입니다. 1992년 총선 때, 김영삼 대선후보는 "집값이 오른 노량진역 부근의 부동산 가격은 세월호 폭침 후 \\\'강남 도시재생\\\' 일환으로 상승했다"는 발언을 했습니다. 하지만 이화선 후보는 이 발언을 "전국적으로 경제적 발전이 이루어지지 않은 지방민의 마음을 멀리해지려는 무례한 발언"이라고 비판하며 문

In [26]:
augmented_data = []
for item in list_data_dict:
    augmented_data.append(item)
    prompt = item['prompt']
    completion_0 = item['completion_0']
    completion_1 = item['completion_1']
    completion_2 = item['completion_2']

    # For prompt augmentation
    for new_prompt in augment_sentence(prompt):
        new_data = item.copy()
        new_data['prompt'] = new_prompt
        augmented_data.append(new_data)

    # For each completion augmentation
    for new_completion_0 in augment_sentence(completion_0):
        new_data = item.copy()
        new_data['completion_0'] = new_completion_0
        augmented_data.append(new_data)
    
    for new_completion_1 in augment_sentence(completion_1):
        new_data = item.copy()
        new_data['completion_1'] = new_completion_1
        augmented_data.append(new_data)
        
    for new_completion_2 in augment_sentence(completion_2):
        new_data = item.copy()
        new_data['completion_2'] = new_completion_2
        augmented_data.append(new_data)

    if len(augmented_data) >= 15000:
        break

In [27]:
print(f"Augmented data size: {len(augmented_data)}")

Augmented data size: 15015


In [37]:
# 지정된 경로 및 파일명
file_path = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_RM_augmented.jsonl'

# 파일 쓰기 모드로 파일 열기
with open(file_path, 'w') as f:
    for item in augmented_data:
        f.write(json.dumps(item) + '\n')

마지막으로 PPO 학습에 쓰일 데이터를 살펴보겠습니다.

In [28]:
data_path_3_PPO = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl'
with open(data_path_3_PPO, "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

print(len(list_data_dict))
list_data_dict[:3]

12000


[{'prompt': '번디는 자신이 탐정잡지, 범죄소설 그리고 성범죄 관련 실제 범죄 다큐멘터리들을 탐독했다고 누구에게 말했나?'},
 {'prompt': '개포주공아파트는 몇 단지로 이루어져 있나?'},
 {'prompt': '김영삼의 후보 시절 지역표심을 겨냥한 발언을 문제삼은 후보는?'}]

In [30]:
augmented_data = []
for item in list_data_dict:
    augmented_data.append(item)
    prompt = item['prompt']

    # 'prompt' 증강
    for new_prompt in augment_sentence(prompt):
        new_data = item.copy()
        new_data['prompt'] = new_prompt
        augmented_data.append(new_data)

    if len(augmented_data) >= 15000:
        break

In [31]:
print(f"Augmented data size: {len(augmented_data)}")

Augmented data size: 15000


In [38]:
# 지정된 경로 및 파일명
file_path = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_PPO_augmented.jsonl'

# 파일 쓰기 모드로 파일 열기
with open(file_path, 'w') as f:
    for item in augmented_data:
        f.write(json.dumps(item) + '\n')

## Supervised Fine-Tuning

### SFT(Supervised Fine-Tuning)
이번 스텝에서는 kogpt-2를 instruction dataset으로 SFT를 진행해 보겠습니다.



In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.optim import Adam
from datasets import load_dataset
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
import copy
import logging
import json
from dataclasses import dataclass

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:
#모델과 토크나이저
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

print(tokenizer)

GPT2TokenizerFast(name_or_path='skt/kogpt2-base-v2', vocab_size=51200, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '</s>', 'eos_token': '</s>', 'unk_token': '</s>', 'pad_token': '</s>'}, clean_up_tokenization_spaces=True)


모델 인퍼런스 단계에서 사용할 prompt 딕셔너리 템플릿과 SFT 데이터셋 클래스를 정의하겠습니다.

In [14]:
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):

    def __init__(self, data_path_1_SFT: str, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        super(SFT_dataset, self).__init__()
        logging.warning("Loading data...")

        pattern_instruction = 'prompt'  # instruction
        pattern_output = 'completion'  # response

        data_path_1_SFT = '/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl'
        with open(data_path_1_SFT, "r", encoding='utf-8-sig') as json_file:
            list_data_dict = json.load(json_file)

        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
            )
        }

        prompt_input = PROMPT_DICT["prompt_input"]

        sources = []
        for example in list_data_dict:
            tmp = prompt_input.format_map(example)
            sources.append(tmp)

        targets = []
        for example in list_data_dict:
            targets.append(f"{example[pattern_output]}{tokenizer.eos_token}")
        examples = [s + t for s, t in zip(sources, targets)]

        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # source
        examples_tokenized = self._tokenize_fn(examples, tokenizer)  # source + target

        input_ids = examples_tokenized["input_ids"]
        labels = copy.deepcopy(input_ids)
        for label, source_len in zip(labels, sources_tokenized["input_ids_lens"]):
            label[:source_len] = -100

        data_dict = dict(input_ids=input_ids, labels=labels)

        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        logging.warning("Loading data done!!: %d"%(len(self.labels)))


    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = [
            tokenizer(
                text,
                return_tensors="pt",
                padding="longest",
                max_length=tokenizer.model_max_length,
                truncation=True,
            )
            for text in strings
        ]
        input_ids = labels = [tokenized.input_ids[0] for tokenized in tokenized_list]
        input_ids_lens = labels_lens = [
            tokenized.input_ids.ne(tokenizer.pad_token_id).sum().item() for tokenized in tokenized_list
        ]
        return dict(
            input_ids=input_ids,
            labels=labels,
            input_ids_lens=input_ids_lens,
            labels_lens=labels_lens,
        )


    def __len__(self):
        return len(self.input_ids)


    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])

In [15]:
@dataclass
class DataCollatorForSupervisedDataset(object): 

    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first=True, padding_value= -100)
        return dict(
            input_ids=input_ids,
            labels=labels,
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )

label[:source_len] = -100은 훈련 중에 어떤 위치의 레이블을 예측하지 않아야 하는지를 지정하기 위해 사용되고, padding_value=-100은 효율적인 배치 처리를 위해 패딩된 시퀀스를 만들 때 사용되며 개별 시퀀스의 내용에는 직접적인 영향을 주지 않습니다. 두 가지 모두 마스킹(masking) 개념과 관련이 있지만, 시퀀스 처리의 다른 단계에서 서로 다른 목적으로 사용됩니다.

이제 SFT_dataset 클래스를 사용해 훈련셋을 만들고 data collator 인스턴스를 만들겠습니다.






In [16]:
train_dataset = SFT_dataset(data_path_1_SFT='/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_1_SFT.jsonl', tokenizer=tokenizer)
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

print('input : %s'%train_dataset.input_ids[0])
print('output: %s'%train_dataset.labels[0])

input : tensor([  739,   378,   378,   378, 14659, 13394, 37091, 10651,   383, 25841,
         8006, 14914,   375,  7673, 20479,  8091, 22311,  9036, 30902, 13675,
          375,   378,   378,   378, 41951,   454,  9549, 20549,   383,  8142,
         7192, 14914,   382, 37767, 13753,  8263,  7166,   739,  8352,  7659,
         9594, 25585, 13600,  8022,  9378, 11532,  9887, 11218,  9111, 16691,
        10351, 10561,  9128, 20479,  8091,  9065,  9446,  9036, 28420, 26521,
        10163, 26367,  6958,  9030,  9882, 12317, 25882,  9209, 37194, 10351,
         9036, 12168, 10529, 15989,  9719, 15434, 10552, 11188, 13362,  9036,
        15805, 11300, 11846,  9146, 16691,  9181,  7397, 15806, 13480, 11342,
        17596,  9161, 19996,  9025, 25006, 18595,  9966, 12592, 10751, 11814,
         8711,  9046, 12450,  9117,  7377, 12521,     1])
output: tensor([ -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,
         -100,  -100,  -100,  -100,  -100,  -100,  -100,  -100,  -10

train_dataset.input_ids[0] 와 train_dataset.labels[0] 를
디코딩하는 함수를 작성하여 정수토큰들의 인코딩전 원래 문장이
어떤 형태로 토크나이징 되었는지 확인해보세요.

In [17]:
# from transformers import AutoTokenizer

# # 모델과 토크나이저를 불러옵니다. 모델 이름은 적절하게 바꿔주세요.
# model_name = "'skt/kogpt2-base-v2'"
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# def decode_input_and_labels(input_ids, label_ids):
#     # input_ids와 label_ids를 텍스트로 디코딩합니다.
#     input_text = tokenizer.decode(input_ids, skip_special_tokens=True)
#     label_text = tokenizer.decode(label_ids, skip_special_tokens=True)
    
#     return input_text, label_text

# # 예제 데이터의 첫 번째 샘플을 디코딩합니다.
# input_text, label_text = decode_input_and_labels(train_dataset.input_ids[0], train_dataset.labels[0])

# # 결과 출력
# print("원래 입력 문장: ", input_text)
# print("레이블 문장: ", label_text)


훈련을 위한 마지막 단계로 Training arguments를 사용해 trainer 클래스를 정의하겠습니다.

In [18]:
training_args = TrainingArguments(
    output_dir="aiffel/KoChatGPT/test",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    warmup_steps=5,
    prediction_loss_only=True,
    fp16 = True
    )
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset
)

In [19]:
trainer.train()
model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_1_SFT')

/opt/conda/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
500,3.283800
1000,3.161500
1500,3.058600
2000,2.996800
2500,2.939500
3000,2.902400
3500,2.786500
4000,2.805500
4500,2.798900
5000,2.715500


이제 문장 생성 능력을 확인하기 위해
빠르게 허깅페이스의 pipleline 클래스를 사용하여 generator를 만들어보겠습니다.

In [21]:
generator = pipeline('text-generation', model='/aiffel/aiffel/KoChatGPT/output_1_SFT', tokenizer=tokenizer)

generation_args = dict(   
    num_beams=4,
    repetition_penalty=2.0,
    no_repeat_ngram_size=4,
    eos_token_id=375, # \n   
    max_new_tokens=64,
    do_sample=True,
    top_k=50,
    early_stopping=True
)

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = ['불고기용 고기 한우에요?',
               '리처드 닉슨이 43대 부통령직을 수행한 년도는?',
               '시카고 오헤어 국제공항은 어디에 있어?',
               '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt' : tmp}) for tmp in list_prompt]

list_result = generator(list_prompt, **generation_args)   
for prompt, result in zip(list_prompt, list_result):
    print()
    print((result[0]['generated_text']))

/opt/conda/lib/python3.9/site-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(



### Instruction(명령어):
불고기용 고기 한우에요?

### Response(응답):'저는 인공지능 어시스턴트이기 때문에 고기를 먹을 수 없습니다. 하지만 일반적으로 불고기는 건강에 좋은 식품이며, 쇠고기의 질을 높이는 데 매우 중요한 역할을 합니다. 따라서 불고기를 먹는 것은 건강에 좋지 않습니다.\n\n만약 불고기가 건강에 좋다고 생각한다면, 불고기를 섭취하는 것은

### Instruction(명령어):
리처드 닉슨이 43대 부통령직을 수행한 년도는?

### Response(응답):'리처드 닉슨은 42대 부통령직을 수행했습니다. "리처드 닉슨"이 어떤 인물인지 알려주시면 더 정확한 답변을 드릴 수 있을 것 같습니다. "이미지"가 무엇을 의미하는지 알려주시면, 더 자세한 답변을 제공해 드릴 수 있을 것입니다. "이미지의"는

### Instruction(명령어):
시카고 오헤어 국제공항은 어디에 있어?

### Response(응답):'시카고 오 헤어 국제공항은 미국 캘리포니아주 샌프란시스코에 위치해 있습니다. Americano, I am an AI language model, I cannot provide more context or information. However, I can assist you ple

### Instruction(명령어):
오늘 미세먼지 어때?

### Response(응답):'저는 인공지능 챗봇이므로, 미세먼지 정보를 알 수 없습니다. 하지만 보통 미세먼지는 대기 중에 발생하는 것으로 알려져 있습니다. 따라서, 해당 지역의 미세먼지 농도를 확인하시는 것이 좋을 것 같습니다. 유용한 정보를 얻으시길 바랍니다. 유용한 정보를 찾으시려면 관련 기관에 문의하


In [22]:
torch.cuda.empty_cache()

## Reward Model

이번 스텝에서는 RLHF의 두번째 단계인 Reward model을 설계하고 학습해보겠습니다.

In [23]:
import os
import json
from typing import Optional
import torch
import torch.nn as nn
from torch.optim import Adam
from chatgpt.dataset import RewardDataset
from chatgpt.models.base import RewardModel
from chatgpt.trainer import RewardModelTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel, AutoConfig
from transformers.models.gpt2.configuration_gpt2 import GPT2Config
from transformers.models.gpt2.modeling_gpt2 import GPT2Model
import loralib as lora

In [24]:
class GPTRM_custom(RewardModel):

    def __init__(self,
                 pretrained: Optional[str] = None,
                 config: Optional[GPT2Config] = None,
                 checkpoint: bool = False,
                 lora_rank: int = 0,
                 lora_train_bias: str = 'none',
                 tokenizer=None) -> None:
        if pretrained is not None:
            model = GPT2Model.from_pretrained(pretrained)
            model.resize_token_embeddings(len(tokenizer))
        elif config is not None:
            model = GPT2Model(config)
        else:
            model = GPT2Model(GPT2Config())
        if checkpoint:
            model.gradient_checkpointing_enable()

        value_head = nn.Linear(model.config.n_embd, 1)
        super().__init__(model, value_head, lora_rank, lora_train_bias)

        if pretrained is not None:
            self.model = model
            self.pretrained = pretrained


    def save_pretrained(self, dir):
        if self.pretrained is not None:
            self.model.save_pretrained(dir)

In [25]:
model = AutoModelForCausalLM.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
    padding_side="right",
    model_max_length=512,
)

with NaiveStrategy().model_init_context():
        model = GPTRM_custom(pretrained='skt/kogpt2-base-v2', lora_rank=0, tokenizer=tokenizer).cuda()

Some weights of the model checkpoint at skt/kogpt2-base-v2 were not used when initializing GPT2Model: ['lm_head.weight']
- This IS expected if you are initializing GPT2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


다음으로 RM을 훈련시킬 때 사용할 ranking dataset을 만들어보겠습니다.

In [27]:
with open('/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_2_RM.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)

total_data_ranking2chosen = []
for tmp in list_data_dict:
    one_data_ranking2chosen = []

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][1]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_1']
    else:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][0] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_0']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_0']
    one_data_ranking2chosen.append(data)

    data = {}
    data['prompt'] = tmp['prompt']
    if tmp['ranking'][1] < tmp['ranking'][2]:
        data['chosen'] = tmp['completion_1']
        data['rejected'] = tmp['completion_2']
    else:
        data['chosen'] = tmp['completion_2']
        data['rejected'] = tmp['completion_1']
    one_data_ranking2chosen.append(data)



    total_data_ranking2chosen.extend(one_data_ranking2chosen)

print('before data num: %d'%(len(list_data_dict)))
print('after  data num: %d'%(len(total_data_ranking2chosen)))
print('data example: \n%s'%total_data_ranking2chosen[45])

before data num: 10220
after  data num: 30660
data example: 
{'prompt': '애플은 리사를 어떻게 처리했어', 'chosen': '애플이 누구인지 명확히 알 수 없어서, 리사가 누구인지와 어떤 상황에서 처리되었는지에 대한 추가적인 정보가 필요합니다. 따라서, 보다 정확한 답변을 제공할 수 없습니다.', 'rejected': '애플은 리사를 위해 고객 서비스 부서에서 고객 다양한 컴퓨터 관련 문제에 대해 응답하는 데 필요한 모든 지원을 제공했습니다. 사용자가 하드웨어 문제를 경험할 때, 전문가들은 필요한 수리(수리, 추가 부품 제공, 소프트웨어 업그레이드 등)을 제공해 드릴 수 있습니다. 또한, 사용자가 사용 방법 문제나 기타 문제를 경험할 때, 대화 상대로 사용자를 지원할 수 있는 전문 고객 서비스 직원들이 사용자에게 상담하고 도움을 주는 데 도움이 될 수 있는 정보를 제공합니다. 또한, 인터넷에서 제공되는 정보를 통해 문제를 해결하거나 고객 서비스 웹 사이트를 통해 자신의 문제를 진단할 수 있도록 하는 등 다양한 방법으로 리사를 처리해 왔습니다.'}


kochatgpt_2_RM.jsonl 은
chatGPT, davinch, ada 세개 모델에 같은 prompt를 주고 얻은 세 답변을
순서대로 good, bad, worst로 간주해
순서를 뒤섞어 completion_0, completion_1, completion_2 세 키에 할당하여 만든 데이터셋입니다.
위와 같이 코드를 짜게 되면 chosen과 resjected에 각각
completion_0, completion_1, completion_2 세개 답변이 가능한 모든 조합으로 들어가게 되어
chosen에 worst 답변이 들어가고
rejected에 good답변이 들어간 데이터도 만들어집니다.

In [28]:
#이제 완성한 ranking dataset을 shuffle한 후 훈련셋을 만들어보겠습니다.
import random
random.seed(230319)
random.shuffle(total_data_ranking2chosen)
print(total_data_ranking2chosen[45])

{'prompt': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은?', 'chosen': '유아인이 류승완 감독을 만나 영화 베테랑의 시나리오를 받았던 곳은 류승완의 사무실입니다.', 'rejected': '대구 영화사옥'}


In [29]:
train_data = total_data_ranking2chosen[:1000] 
eval_data = total_data_ranking2chosen[1000:1200]

print(len(train_data))
print(len(eval_data))

train_dataset = RewardDataset(train_data, tokenizer, 512)
eval_dataset = RewardDataset(eval_data, tokenizer, 512)

1000
200


100%|██████████| 200/200 [00:00<00:00, 1060.06it/s]


In [30]:
idx = 1
print('#'*70)
print('## prompt ##')
print(train_data[idx]['prompt'])
print('#'*70)
print('## chosen ##')
print(train_data[idx]['chosen'])
print('#'*70)
print('## rejected ##')
print(train_data[idx]['rejected'])

######################################################################
## prompt ##
흑고래의 무게는 어느 정도야
######################################################################
## chosen ##
흑고래의 평균 몸무게는 약 25~40톤 정도이지만, 최대 몸무게는 50톤 이상에 이를 수 있습니다.
######################################################################
## rejected ##
흑고래의 무게는 매우 다양하게 달라집니다. 약 200kg에서 10톤까지 달라질 수 있습니다.


In [34]:
trainer = RewardModelTrainer(model=model,
                             strategy=NaiveStrategy(),
                             optim=Adam(model.parameters(), lr=5e-5),
                             train_dataset=train_dataset,
                             eval_dataset=eval_dataset,
                             batch_size=2,
                             max_epochs=1)

In [35]:
trainer.fit(use_lora=0)

model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_2_RM')





Train epoch:   0%|          | 0/1 [00:00<?, ?it/s]




Train step of epoch 0:   0%|          | 0/500 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(use_lora=0)                                                                      │
│   2                                                                                              │
│   3 model.save_pretrained('/aiffel/aiffel/KoChatGPT/output_2_RM')                                │
│   4                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/chatgpt/trainer/rm.py:65 in fit                           │
│                                                                                                  │
│   62 │   │   │   │   c_mask = c_mask.squeeze(1).cuda()                                           │
│   63 │   │   │   │   reject_ids = reject_ids.squeeze(1).cuda()                                   │
│   64 │   │   │   │   r_mask = r_mask.squeeze(1).cuda()                                           │
│ ❱ 65 │   │   │   │   chosen_reward = self.model(chosen_ids, attention_mask=c_mask)               │
│   66 │   │   │   │   reject_reward = self.model(reject_ids, attention_mask=r_mask)               │
│   67 │   │   │   │   loss = self.loss_fn(chosen_reward, reject_reward)                           │
│   68 │   │   │   │   self.strategy.backward(loss, self.model, self.optimizer)                    │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                                                                  │
│   1127 │   │   # this function, and just call forward.                                           │
│   1128 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1129 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1130 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1131 │   │   # Do not call functions when jit is used                                          │
│   1132 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1133 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/chatgpt/models/base/reward_model.py:37 in forward         │
│                                                                                                  │
│   34 │   │   │   self.value_head = nn.Linear(model.config.n_embd, 1)                             │
│   35 │                                                                                           │
│   36 │   def forward(self, sequences: torch.LongTensor, attention_mask: Optional[torch.Tensor    │
│ ❱ 37 │   │   outputs = self.model(sequences, attention_mask=attention_mask)                      │
│   38 │   │   last_hidden_states = outputs['last_hidden_state']                                   │
│   39 │   │   values = self.value_head(last_hidden_states)[:, :-1]                                │
│   40 │   │   value = values.mean(dim=1).squeeze(1)    # ensure shape is (B)                      │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/torch/nn/modules/module.py:1130 in _call_impl             │
│                                                            

RM 학습이 잘 되었는지 확인해보기 위해 임의의 문장을 입력한 후
적절한 reward score를 출력하는지 살펴보도록 하겠습니다.

In [ ]:
def inference_RM(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    output = model(input_ids)
    output_reward = output.cpu().detach().numpy()[0]

    print('input: %s\nreward score: %.1f'%(input_text, output_reward))

    return output_reward

input_text = '인공지능은 똥멍청이 입니다'
output_reward = inference_RM(input_text=input_text)

In [ ]:
input_text = '인공지능(AI)은 컴퓨터에서 음성 및 작성된 언어를 보고 이해하고 번역하고 데이터를 분석하고 추천하는 기능을 포함하여 다양한 고급 기능을 수행할 수 있는 일련의 기술입니다.'

output_reward = inference_RM(input_text=input_text)

In [ ]:
torch.cuda.empty_cache()

## Proximal Policy Optimization


In [36]:
from copy import deepcopy

import torch
from torch.optim import Adam
from chatgpt.models.base import RewardModel
from chatgpt.models.gpt import GPTActor, GPTCritic
from chatgpt.trainer import PPOTrainer
from chatgpt.trainer.strategies import NaiveStrategy
from transformers import AutoTokenizer

노드에서 소개하는 KoChatGPT의 경우
PPO에 사용할 actor모델은 1단계 SFT 모델을,
critic모델은 2단계 RM 모델을 사용합니다.

그리고 actor 모델이 critic 모델로부터 피드백을 받아 파라미터를 업데이트 할 때
적절한 페널티를 줄 수 있도록 하는 initial model은
SFT모델을 그대로 freezing 하여 사용합니다.

토크나이저는 pretrain 모델인 kogpt-2의 토크나이저를 그대로 사용해야겠죠?



In [38]:
with NaiveStrategy().model_init_context():
    actor = GPTActor(pretrained='/aiffel/aiffel/KoChatGPT/output_1_SFT', lora_rank=0).to(torch.cuda.current_device())
    critic = GPTCritic(pretrained='/aiffel/aiffel/KoChatGPT/output_2_RM', lora_rank=0).to(torch.cuda.current_device())

    tokenizer = AutoTokenizer.from_pretrained(
        'skt/kogpt2-base-v2', bos_token='</s>', eos_token='</s>', unk_token='</s>', pad_token='</s>',
        padding_side="right", 
        model_max_length=512
    )

    initial_model = deepcopy(actor)
    reward_model = RewardModel(deepcopy(critic.model), deepcopy(critic.value_head)).to(torch.cuda.current_device())

In [39]:
actor_optim = Adam(actor.parameters(), lr=5e-6)
critic_optim = Adam(critic.parameters(), lr=5e-6)

In [40]:
(actor, actor_optim), (critic, critic_optim), reward_model, initial_model = NaiveStrategy().prepare(
    (actor, actor_optim), (critic, critic_optim), reward_model, initial_model)

PPO 학습에 쓸 데이터를 불러와 토크나이징 해줍니다.

In [43]:
with open('/aiffel/aiffel/KoChatGPT/data_kochatgpt/kochatgpt_3_PPO.jsonl', "r", encoding='utf-8-sig') as json_file:
    list_data_dict = json.load(json_file)
    list_prompt = [tmp['prompt'] for tmp in list_data_dict]

def tokenize_fn(texts):
    batch = tokenizer(texts, return_tensors='pt', max_length=96, padding=True, truncation=True)
    return {k: v.cuda() for k, v in batch.items()}

In [44]:
print(tokenize_fn('It takes something more than intelligence to act intelligently.'))

{'input_ids': tensor([[47311, 10448, 19008,  9792, 11780, 11308, 30190, 10929, 11849, 21663,
         44389,  9574, 13799,   458, 14308, 12778, 22469, 20938, 44696,   458,
         13799,   458, 14308, 12778, 11756, 18944,   389]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1]], device='cuda:0')}


In [45]:
len(list_prompt)

12000

PPO는 별도의 PPOTrainer 클래스를 설계하여 학습시켜줘야 합니다.
빠르게 실습해보기 위해 1epoch만 돌려보겠습니다.

In [50]:
trainer = PPOTrainer(NaiveStrategy(),
                     actor,
                     critic,
                     reward_model,
                     initial_model,
                     actor_optim,
                     critic_optim,
                     max_epochs=1,  
                     train_batch_size=2, 
                     tokenizer=tokenize_fn,
                     max_length=128,
                     do_sample=True,
                     temperature=1.0,
                     top_k=50,
                     pad_token_id=tokenizer.pad_token_id,
                     eos_token_id=tokenizer.eos_token_id)

In [51]:
trainer.fit(list_prompt, 
            num_episodes=10,  
            max_timesteps=3,
            update_timesteps=3)

model.save_pretrained('aiffel/KoChatGPT/output_3_PPO')







Train step of epoch 0:   0%|          | 0/500 [14:45<?, ?it/s]


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 trainer.fit(list_prompt,                                                                     │
│   2 │   │   │   num_episodes=10,                                                                 │
│   3 │   │   │   max_timesteps=3,                                                                 │
│   4 │   │   │   update_timesteps=3)                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/chatgpt/trainer/base.py:114 in fit                        │
│                                                                                                  │
│   111 │   │   │   │   else:                                                                      │
│   112 │   │   │   │   │   inputs = rand_prompts                                                  │
│   113 │   │   │   │   self._on_make_experience_start()                                           │
│ ❱ 114 │   │   │   │   experience = self._make_experience(inputs)                                 │
│   115 │   │   │   │   self._on_make_experience_end(experience)                                   │
│   116 │   │   │   │   self.replay_buffer.append(experience)                                      │
│   117 │   │   │   │   if time % update_timesteps == 0:                                           │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/chatgpt/trainer/base.py:64 in _make_experience            │
│                                                                                                  │
│    61 │   │   if isinstance(inputs, Tensor):                                                     │
│    62 │   │   │   return self.experience_maker.make_experience(inputs, **self.generate_kwargs)   │
│    63 │   │   elif isinstance(inputs, dict):                                                     │
│ ❱  64 │   │   │   return self.experience_maker.make_experience(**inputs, **self.generate_kwarg   │
│    65 │   │   else:                                                                              │
│    66 │   │   │   raise ValueError(f'Unsupported input type "{type(inputs)}"')                   │
│    67                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.9/site-packages/chatgpt/experience_maker/naive.py:19 in make_experience   │
│                                                            

드디어 SFT, RM 그리고 PPO 학습이 모두 완료되었습니다.
RLHF가 적용된 koGPT-2의 생성능력을 확인해볼까요?

In [ ]:
def generation(input_text):
    input_ids = tokenizer.encode(input_text, return_tensors='pt').to(
        torch.cuda.current_device())
    outputs = actor.generate(input_ids,
                             max_length=250,
                             do_sample=True,
                             top_k=50,
                             top_p=0.95,
                             num_return_sequences=1)
    output = tokenizer.batch_decode(outputs[0], skip_special_tokens=True)[0]
    print()
    print(output)
    return output

PROMPT_DICT = {
    "prompt_input": (
        "### Instruction(명령어):\n{prompt}\n\n### Response(응답):"
    )
}

list_prompt = [
    '불고기용 고기 한우에요?', 
    '리처드 닉슨이 43대 부통령직을 수행한 년도는?', 
    '시카고 오헤어 국제공항은 어디에 있어',
    '오늘 미세먼지 어때?']

list_prompt = [PROMPT_DICT['prompt_input'].format_map({'prompt': tmp}) for tmp in list_prompt]

for input_text in list_prompt:
    output = generation(input_text)

## 회고
- 강화학습은 처음이라, 아직 학습 노드 내용을 이해하지 못했다. 
- 요즘 계속 CUDA OOM 에러가 떠서 학습이 잘 안되고 있다.
- 이번 프로젝트는 가이드가 적어서 특히 더 어려웠다. 수행해야 하는 task 3개 중에서 데이터 전처리 정도만 수행해 볼 수 있었다.  
  시간이 오래 걸리더라도, 나머지 두개의 task에 대해서도 학습이 필요하다.